---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the Jupyter Notebook FAQ course resource._

---

# Assignment 4 - Predicting and understanding viewer engagement with educational videos 

With the accelerating popularity of online educational experiences, the role of online lectures and other educational video continues to increase in scope and importance. Open access educational repositories such as <a href="http://videolectures.net/">videolectures.net</a>, as well as Massive Open Online Courses (MOOCs) on platforms like Coursera, have made access to many thousands of lectures and tutorials an accessible option for millions of people around the world. Yet this impressive volume of content has also led to a challenge in how to find, filter, and match these videos with learners. This assignment gives you an example of how machine learning can be used to address part of that challenge.

## About the prediction problem

One critical property of a video is engagement: how interesting or "engaging" it is for viewers, so that they decide to keep watching. Engagement is critical for learning, whether the instruction is coming from a video or any other source. There are many ways to define engagement with video, but one common approach is to estimate it by measuring how much of the video a user watches. If the video is not interesting and does not engage a viewer, they will typically abandon it quickly, e.g. only watch 5 or 10% of the total. 

A first step towards providing the best-matching educational content is to understand which features of educational material make it engaging for learners in general. This is where predictive modeling can be applied, via supervised machine learning. For this assignment, your task is to predict how engaging an educational video is likely to be for viewers, based on a set of features extracted from the video's transcript, audio track, hosting site, and other sources.

We chose this prediction problem for several reasons:

* It combines a variety of features derived from a rich set of resources connected to the original data;
* The manageable dataset size means the dataset and supervised models for it can be easily explored on a wide variety of computing platforms;
* Predicting popularity or engagement for a media item, especially combined with understanding which features contribute to its success with viewers, is a fun problem but also a practical representative application of machine learning in a number of business and educational sectors.


## About the dataset

We extracted training and test datasets of educational video features from the VLE Dataset put together by researcher Sahan Bulathwela at University College London. 

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single educational video, and includes information about diverse properties of the video content as described further below. The target variable is `engagement` which was defined as True if the median percentage of the video watched across all viewers was at least 30%, and False otherwise.

Note: Any extra variables that may be included in the training set are simply for your interest if you want an additional source of data for visualization, or to enable unsupervised and semi-supervised approaches. However, they are not included in the test set and thus cannot be used for prediction. **Only the data already included in your Coursera directory can be used for training the model for this assignment.**

For this final assignment, you will bring together what you've learned across all four weeks of this course, by exploring different prediction models for this new dataset. In addition, we encourage you to apply what you've learned about model selection to do hyperparameter tuning using training/validation splits of the training data, to optimize the model and further increase its performance. In addition to a basic evaluation of model accuracy, we've also provided a utility function to visualize which features are most and least contributing to the overall model performance.

**File descriptions** 
    assets/train.csv - the training set (Use only this data for training your model!)
    assets/test.csv - the test set
<br>

**Data fields**

train.csv & test.csv:

    title_word_count - the number of words in the title of the video.
    
    document_entropy - a score indicating how varied the topics are covered in the video, based on the transcript. Videos with smaller entropy scores will tend to be more cohesive and more focused on a single topic.
    
    freshness - The number of days elapsed between 01/01/1970 and the lecture published date. Videos that are more recent will have higher freshness values.
    
    easiness - A text difficulty measure applied to the transcript. A lower score indicates more complex language used by the presenter.
    
    fraction_stopword_presence - A stopword is a very common word like 'the' or 'and'. This feature computes the fraction of all words that are stopwords in the video lecture transcript.
    
    speaker_speed - The average speaking rate in words per minute of the presenter in the video.
    
    silent_period_rate - The fraction of time in the lecture video that is silence (no speaking).
    
train.csv only:
    
    engagement - Target label for training. True if learners watched a substantial portion of the video (see description), or False otherwise.
    

## Evaluation

Your predictions will be given as the probability that the corresponding video will be engaging to learners.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model with an AUC (area under ROC curve) of at least 0.8 passes this assignment, and over 0.85 will receive full points.
___

For this assignment, create a function that trains a model to predict significant learner engagement with a video using `asset/train.csv`. Using this model, return a Pandas Series object of length 2309 with the data being the probability that each corresponding video from `readonly/test.csv` will be engaging (according to a model learned from the 'engagement' label in the training set), and the video index being in the `id` field.

Example:

    id
       9240    0.401958
       9241    0.105928
       9242    0.018572
                 ...
       9243    0.208567
       9244    0.818759
       9245    0.018528
             ...
       Name: engagement, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out and check your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. 

* Try to avoid global variables. If you have other functions besides engagement_model, you should move those functions inside the scope of engagement_model.

* Be sure to first check the pinned threads in Week 4's discussion forum if you run into a problem you can't figure out.

### Extensions

* If this prediction task motivates you to explore further, you can find more details here on the original VLE dataset and others related to video engagement: https://github.com/sahanbull/VLE-Dataset



In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)   # Do not change this value: required to be compatible with solutions generated by the autograder.

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
X_test1 = df_test.iloc[:,1:9]
X_train1 = df_train.iloc[:,1:9]
y_train1 = df_train.iloc[:,-1]
X_test1

,title_word_count,document_entropy,freshness,easiness,fraction_stopword_presence,normalization_rate,speaker_speed,silent_period_rate
0,6,8.548351,14140,89.827395,0.640810,0.017945,2.262723,0.000000
1,8,7.730110,14600,82.446667,0.606738,0.027708,2.690351,0.000000
2,3,8.200887,16980,88.821542,0.621089,0.009857,3.116071,0.000000
3,5,6.377299,16260,86.874660,0.600000,0.004348,2.837500,0.017994
4,18,7.756530,14030,88.872277,0.616105,0.033240,1.354839,0.000000
...,...,...,...,...,...,...,...,...
2304,11,7.781813,13920,80.620236,0.611722,0.031276,2.193125,0.194918
2305,3,7.800766,13990,94.080225,0.654741,0.013967,1.580000,0.338601
2306,2,8.235828,14280,95.173070,0.618686,0.018486,5.548518,0.369348
2307,13,6.775492,15900,74.053098,0.560472,0.032448,2.592308,0.005131


In [3]:
#Let's also make a test data from our own train set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, random_state=0)

In [4]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

#We will use a Scaler

scaler = MinMaxScaler()
#To the train and test set from the train original
X_train_es = scaler.fit_transform(X_train)
X_test_es = scaler.transform (X_test)
#Scaler for the orginal data
X_test1_es = scaler.transform (X_test1)

#Seeing that the data is unbalanced, let's use compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

#Finally let's make the model
svm_model = SVC(kernel='linear', class_weight=dict(enumerate(class_weights)), probability=True)

In [5]:
#With this model, let's see which value for C is better
from sklearn.model_selection import GridSearchCV

param_grid = {'C': np.arange(5, 10, 0.5)}
grid_search = GridSearchCV(estimator = svm_model, param_grid=param_grid, cv = 3, scoring='accuracy')
grid_search.fit(X_train_es, y_train)
print('best_score:',  grid_search.best_score_,
        'best_params:' , grid_search.best_params_)

best_score: 0.8217635114993279 best_params: {'C': 7.5}


In [6]:
# Now we know the best model
best_model = grid_search.best_estimator_
best_model
y_pred_proba = best_model.decision_function(X_test1_es)


In [7]:
#Let's see the AUC roc
from sklearn.metrics import  roc_auc_score
y_pred_proba = best_model.decision_function(X_test_es)
auc_roc = roc_auc_score(y_test, y_pred_proba)

print("AUC-ROC:", auc_roc)

AUC-ROC: 0.8547049717569949


In [10]:
#Finally we obtain the probability for the real test set
y_pred_proba_final = best_model.predict_proba(X_test1_es)[:,1]

df_final = pd.Series(y_pred_proba_final, index=df_test['id'], name='engagement')
df_final

id
9240     0.015503
9241     0.076722
9242     0.102697
9243     0.680782
9244     0.028940
           ...   
11544    0.037430
11545    0.026350
11546    0.020661
11547    0.443739
11548    0.049491
Name: engagement, Length: 2309, dtype: float64